# Object Creation: Notification Objects

This notebook demonstrates how to use the NotificationEventManager class to process notification events and create notification objects from OCED data.

### Notes:
- Update the file paths and user ID to match your data
- The notification objects track the last action (RECEIVED/READ) and its timestamp
- Each notification is linked to both the user and the day it occurred
- The extended data includes both the original OCED data and the new notification objects

## 1. Setup

In [2]:
import datetime
import sys
# Add the project root directory to the Python path
sys.path.append('..')
from concurrent.futures import ThreadPoolExecutor
from src.oced.oced_data_query import OCEDDataQuery
from src.oced.time_objects import TimeObject
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats
import gc
import json
import pandas as pd
from pathlib import Path
from typing import Dict, Any
from src.oced.notification_events import NotificationEventManager

## 2. Load OCED Data

In [3]:
# Get OCED-mHealth Data from JSON File
dataQuery = OCEDDataQuery()  
oced_data_file = f"oced_data_time_bouts.json"
data_dict = dataQuery.load_json(oced_data_file)

# Quick look at the data structure
print(f"Number of behavior events: {len(data_dict.get('behaviorEvents', []))}")
print(f"Number of objects: {len(data_dict.get('objects', []))}")

Number of behavior events: 134
Number of objects: 63


## 3. Initialize Notification Manager and Process Data

In [ ]:
# Initialize the manager
notification_manager = NotificationEventManager()

# Add notification object type to the data
oced_data = notification_manager.create_notification_object_type(data_dict)

# Create notification objects and link events
# Replace 'your_user_id' with actual user ID
extended_data, notification_objects = notification_manager.create_notification_objects(
    data=oced_data,
    user_id='your_user_id'
)

## 4. Analyse results

In [ ]:
# Print basic statistics
print(f"Total notification objects created: {len(notification_objects)}")

# Count notifications by last action
action_counts = {}
for obj in notification_objects:
    last_action = next(
        (attr['value'] for attr in obj['attributes'] 
         if attr['name'] == 'last_action'),
        None
    )
    action_counts[last_action] = action_counts.get(last_action, 0) + 1

print("\nNotifications by last action:")
for action, count in action_counts.items():
    print(f"- {action}: {count}")

# Optional: Create a DataFrame for more analysis
notifications_df = pd.DataFrame([
    {
        'notification_id': obj['id'],
        'last_action': next((attr['value'] for attr in obj['attributes'] 
                            if attr['name'] == 'last_action'), None),
        'last_action_time': next((attr['time'] for attr in obj['attributes'] 
                                if attr['name'] == 'last_action'), None)
    }
    for obj in notification_objects
])

# Display first few notifications
notifications_df.head()

## 5. Save Processed Data

In [ ]:
# Save the extended data
output_file = "notifications_processed.json"  # Update this name as needed
notification_manager.save_extended_data(
    filename=output_file,
    extended_data=extended_data,
    compress=False  # Set to True for compressed output
)

## 6. Example: Query Notifications for a Specific Day

In [4]:
# Get notifications for a specific day
date_str = "2024-01-01"  # Update this date
day_notifications = notification_manager.get_notifications_for_day(
    date_str=date_str,
    data=extended_data
)

print(f"\nNotifications for {date_str}:")
for notif in day_notifications:
    last_action = next((attr['value'] for attr in notif['attributes'] 
                       if attr['name'] == 'last_action'), None)
    last_action_time = next((attr['time'] for attr in notif['attributes'] 
                            if attr['name'] == 'last_action'), None)
    print(f"- ID: {notif['id']}, Last Action: {last_action} at {last_action_time}")

NameError: name 'notification_manager' is not defined

## 7. Example: Get Specific Notification Details

In [ ]:
# Get details for a specific notification
notification_id = notification_objects[0]['id']  # Example: first notification
notification = notification_manager.get_notification_object(notification_id)

if notification:
    print(f"\nDetails for notification {notification_id}:")
    for attr in notification['attributes']:
        print(f"- {attr['name']}: {attr['value']} (at {attr['time']})")
    print("\nRelationships:")
    for rel in notification['relationships']:
        print(f"- {rel['qualifier']}: {rel['id']} ({rel['type']})")